In [1]:
import sources.logins as lg
from sqlite3 import connect
import sources.parts_list_0802 as pl
import pandas as pd
from datetime import datetime as dt
import xlwings as xw

from datetime import datetime, timedelta
from dateutil import parser

In [ ]:
# check all table
pd.read_sql("SELECT name FROM sqlite_master WHERE type ='table'",conn)

In [ ]:
#call header from m_list
cursor = conn.execute('select * from m_list')
names = list(map(lambda x: x[0], cursor.description))
names
for name in names: print(name)

In [2]:
conn = connect('quotation.db')
cur = conn.cursor()

In [4]:
from datetime import datetime, timedelta
from dateutil import parser

today = datetime.today()
current_weekday = today.weekday()  # 0 for Monday, 1 for Tuesday, ..., 6 for Sunday
monday_of_this_week = today - timedelta(days=current_weekday)

# print("Monday of this week:", monday_of_this_week.date())
ans = str(input(f'Start Date of Report(YYYY-MM-DD): {monday_of_this_week.date()}?[Y]/N') or monday_of_this_week.date())
print (ans)

Start Date of Report(YYYY-MM-DD): 2023-09-25?[Y]/N
2023-09-25


In [ ]:
ans

In [ ]:
from dateutil import parser

# Input date string
date_string = ans

try:
    parsed_date = parser.parse(date_string)
    detected_format = parsed_date.strftime("%Y-%m-%d")
    
    if detected_format == date_string:
        print(f"Valid date in format: {detected_format}")
    else:
        print(f"Valid date, but in a different format: {detected_format}")
except ValueError:
    print("Invalid date format")



In [5]:
# receive this week
report_start_date = ans
q = f'''
        SELECT [rma no.],customer_name,model,serial_no,
        strftime('%Y-%m-%d',recieve_date)AS [receive date],
        recieve_user_name,repair_status,e.location
        
        FROM consolidated c
        LEFT JOIN engineers e ON c.recieve_user_name = e.exfm_name
        WHERE recieve_date >= '{report_start_date}'
        ORDER BY location,customer_name
        
    '''
receive = pd.read_sql(q,conn)
receive

,RMA No.,CUSTOMER_NAME,MODEL,SERIAL_NO,receive date,RECIEVE_USER_NAME,REPAIR_STATUS,location
0,FMSV2023090061,FTYO - Loaner Asset,EG-600WR,2G391K630,2023-09-27,Nguyen Thai,Authorization,HCM
1,FMSV2023090060,Vinh Duc General Hospital,EG-530WR,8G361K202,2023-09-26,Nguyen Thai,Authorization,HCM
2,FMSV2023090057,FTYO - Loaner Asset,EC-760R-V/I,2C728K102,2023-09-25,Minh Duc Nguyen (Hanoi),Authorization,Hanoi
3,FMSV2023090055,"FUJIFILM Vietnam Co., Ltd.",VP-7000,2v644k259,2023-09-25,Minh Duc Nguyen (Hanoi),Authorization,Hanoi
4,FMSV2023090056,"FUJIFILM Vietnam Co., Ltd.",BL-7000,3S101K332,2023-09-25,Minh Duc Nguyen (Hanoi),Authorization,Hanoi
5,FMSV2023090058,"FUJIFILM Vietnam Co., Ltd.",EP-6000,1V696K004,2023-09-26,Quynh Ngan Luu,Receive,Hanoi
6,FMSV2023090054,Hanoi Oncology Hospital,EC-530WL3,KC643K241,2023-09-25,Minh Duc Nguyen (Hanoi),Receive,Hanoi
7,FMSV2023090059,Hung Viet Cancer Hospital,EG-760Z,8G403K185,2023-09-26,Quynh Ngan Luu,Receive,Hanoi


In [6]:
# inspection and repair by location

q = f'''
            SELECT 
                DISTINCT c.[rma no.],customer_name,model,serial_no,recieve_date,in_inspect_date,
                r.[start time],r.[end time],'Inspection' AS [repair size],
                in_inspect_user_name AS PIC,e.location

                FROM (consolidated c
                LEFT JOIN engineers e ON c.recieve_user_name = e.exfm_name)
                LEFT JOIN repair_code r ON c.[rma no.] = r.[rma no.]
                WHERE in_inspect_date >= '{report_start_date}'
                
                
            UNION ALL
                SELECT DISTINCT c.[rma no.],customer_name,model,serial_no,recieve_date,in_inspect_date,
                    r.[start time],r.[end time],c.[repair size],
                    CASE
                        WHEN r.[start user] NOT NULL THEN r.[start user]
                        ELSE c.[repair user name] END AS [Repair User],
                    e.location


                    FROM (consolidated c
                    LEFT JOIN engineers e ON c.recieve_user_name = e.exfm_name)
                    LEFT JOIN repair_code r ON c.[rma no.] = r.[rma no.]
                    WHERE r.[start time] >= '{report_start_date}'
                    
    '''
wr = pd.read_sql(q,conn)
# wr

In [7]:
wr.to_sql('weekly_report',conn,if_exists = 'replace',index = False)

16

In [ ]:
# pd.read_sql('select * from weekly_report ORDER BY location,[repair size]',conn)

In [8]:
# repair type
q = f'''
        SELECT [rma no.],customer_name,model,serial_no,recieve_date,[repair size],location,
        CASE
            WHEN [repair size] = 'Inspection' AND PIC ='Nguyen Thai' THEN 'Inspection'
            WHEN [repair size] = 'Minor' AND PIC ='Nguyen Thai' THEN 'Minor'
            WHEN [repair size] = 'Other' AND PIC ='Nguyen Thai' THEN 'Minor'
            WHEN [repair size] = 'Major' AND PIC ='Nguyen Thai' THEN 'Major'
            
            ELSE '-' END AS 'Nguyen',
        CASE
            WHEN [repair size] = 'Inspection' AND PIC ='hoang' THEN 'Inspection'
            WHEN [repair size] = 'Minor' AND PIC ='hoang' THEN 'Minor'
            WHEN [repair size] = 'Other' AND PIC ='hoang' THEN 'Minor'
            WHEN [repair size] = 'Major' AND PIC ='hoang' THEN 'Major'
            ELSE '-' END AS 'Hoang',
        CASE
            WHEN [repair size] = 'Inspection' AND PIC ='Le Quang Thong' THEN 'Inspection'
            WHEN [repair size] = 'Minor' AND PIC ='Le Quang Thong' THEN 'Minor'
            WHEN [repair size] = 'Other' AND PIC ='Le Quang Thong' THEN 'Minor'
            WHEN [repair size] = 'Major' AND PIC ='Le Quang Thong' THEN 'Major'
            ELSE '-' END AS 'Thong',
        CASE
            WHEN [repair size] = 'Inspection' AND PIC ='Nguyen Khac Thang (Hanoi)' THEN 'Inspection'
            WHEN [repair size] = 'Minor' AND PIC ='Nguyen Khac Thang (Hanoi)' THEN 'Minor'
            WHEN [repair size] = 'Other' AND PIC ='Nguyen Khac Thang (Hanoi)' THEN 'Minor'
            WHEN [repair size] = 'Major' AND PIC ='Nguyen Khac Thang (Hanoi)' THEN 'Major'
            ELSE '-' END AS 'Thang',
        CASE
            WHEN [repair size] = 'Inspection' AND PIC ='Le Van Hoan' THEN 'Inspection'
            WHEN [repair size] = 'Minor' AND PIC ='Le Van Hoan' THEN 'Minor'
            WHEN [repair size] = 'Other' AND PIC ='Le Van Hoan' THEN 'Minor'
            WHEN [repair size] = 'Major' AND PIC ='Le Van Hoan' THEN 'Major'
            ELSE '-' END AS 'Hoanle',
        CASE
            WHEN [repair size] = 'Inspection' AND PIC ='Nguyen Tuan Minh' THEN 'Inspection'
            WHEN [repair size] = 'Minor' AND PIC ='Nguyen Tuan Minh' THEN 'Minor'
            WHEN [repair size] = 'Other' AND PIC ='Nguyen Tuan Minh' THEN 'Minor'
            WHEN [repair size] = 'Major' AND PIC ='Nguyen Tuan Minh' THEN 'Major'
            ELSE '-' END AS 'Minh'
            
            
            
        FROM weekly_report
        ORDER BY location,[repair size]
    '''
completed = pd.read_sql(q,conn)

In [9]:
# open template
wb = xw.Book('templates/weekly_report.xlsx')
sh_in = wb.sheets('In')
sh_out = wb.sheets('Out')

#initial IN
sh_in.range('A2').value = 'Customer'
sh_in.range('B2').value = 'Model'
sh_in.range('C2').value = 'Serial'
sh_in.range('D2').value = 'Recieve'
sh_in.range('E2').value = 'Location'
sh_in.range('F2').value = 'Status'

for i in range(len(receive)):
    sh_in.range('A3').offset(i,0).value = receive['CUSTOMER_NAME'][i]
    sh_in.range('B3').offset(i,0).value = receive['MODEL'][i]
    sh_in.range('C3').offset(i,0).value = receive['SERIAL_NO'][i]
    sh_in.range('D3').offset(i,0).value = receive['receive date'][i]
    sh_in.range('E3').offset(i,0).value = receive['location'][i]
    sh_in.range('F3').offset(i,0).value = receive['REPAIR_STATUS'][i]
    
#initial OUT
sh_out.range('A2').value = 'Customer'
sh_out.range('B2').value = 'Model'
sh_out.range('C2').value = 'Serial'
sh_out.range('D2').value = 'Receive'
sh_out.range('E2').value = 'Nguyên'
sh_out.range('F2').value = 'Hoàng'
sh_out.range('G2').value = 'Thông'

hcm = completed[completed['location']=='HCM']
hcm = hcm.reset_index(drop=True)

for i in range(len(hcm)):
    sh_out.range('A3').offset(i,0).value = hcm['CUSTOMER_NAME'][i]
    sh_out.range('B3').offset(i,0).value = hcm['MODEL'][i]
    sh_out.range('C3').offset(i,0).value = hcm['SERIAL_NO'][i]
    sh_out.range('D3').offset(i,0).value = hcm['RECIEVE_DATE'][i]
    sh_out.range('E3').offset(i,0).value = hcm['Nguyen'][i]
    sh_out.range('F3').offset(i,0).value = hcm['Hoang'][i]
    sh_out.range('G3').offset(i,0).value = hcm['Thong'][i]
 
k = len(hcm)+3

sh_out.range('A2').offset(k,0).value = 'Customer'
sh_out.range('B2').offset(k,0).value = 'Model'
sh_out.range('C2').offset(k,0).value = 'Serial'
sh_out.range('D2').offset(k,0).value = 'Receive'
sh_out.range('E2').offset(k,0).value = 'Thắng'
sh_out.range('F2').offset(k,0).value = 'Hoàn'
sh_out.range('G2').offset(k,0).value = 'Minh'

hanoi = completed[completed['location']=='Hanoi']
hanoi = hanoi.reset_index(drop=True)

for i in range(len(hanoi)):
    sh_out.range('A3').offset(i+k,0).value = hanoi['CUSTOMER_NAME'][i]
    sh_out.range('B3').offset(i+k,0).value = hanoi['MODEL'][i]
    sh_out.range('C3').offset(i+k,0).value = hanoi['SERIAL_NO'][i]
    sh_out.range('D3').offset(i+k,0).value = hanoi['RECIEVE_DATE'][i]
    sh_out.range('E3').offset(i+k,0).value = hanoi['Thang'][i]
    sh_out.range('F3').offset(i+k,0).value = hanoi['Hoanle'][i]
    sh_out.range('G3').offset(i+k,0).value = hanoi['Minh'][i]
print('Done')

Done


In [10]:
from datetime import datetime as dt
import os
today = dt.now().strftime('%y%m%d')
folder_name = 'weekly_report'
try:
    os.mkdir(folder_name)
    print(f'folder {folder_name} was created.')
    
except:
    print(f'Folder {folder_name} exists')
try:
    wb.save(f'{folder_name}/weekly_report_{today}.xlsx')
    print(f'{wb.name} saved and close.')
    wb.close()
except:
    pass

Folder weekly_report exists
weekly_report_230928.xlsx saved and close.


In [ ]:
completed

In [ ]:
wb.close()